In [1]:
import pandas as pd
import psycopg2
import numpy as np



from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service



from tqdm import tqdm
tqdm.pandas()
import time



from bs4 import BeautifulSoup
import re

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders


In [2]:
s = Service("../chromedriver_win32/chromedriver.exe")

In [3]:
class postgres_conn:
    def getConn(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='postgres_007',
                                          host="1.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error
    def getConn2(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='reporting_007',
                                          host="2.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error


    def close_connection(self, cursor, connection):
        # closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
pg = postgres_conn()
conn = pg.getConn()
conn2 = pg.getConn2()

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '1.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '2.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [4]:
job_id=input("Enter keystone ids with comma.").split(",")
days=input(" Enter no of days for which data needs brand mapping.\n")
job_id.append("")
tuple(job_id)

Enter keystone ids with comma.649578a677c5295142c2c620
 Enter no of days for which data needs brand mapping.
50


('649578a677c5295142c2c620', '')

In [5]:
query = f"""
select brand,title,sku,job_name,channel_id
from ready.serp_details smd join ready.serp_mongo_details m on m.serp_mongo_id=smd.serp_mongo_id
where smd.crawl_date > now() - interval '{days} day' and keystone_job_id in {tuple(job_id)}
"""
serp_details = pd.read_sql_query(query, conn[1])

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_157092\2162843895.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  serp_details = pd.read_sql_query(query, conn[1])


In [6]:
serp_details

,brand,title,sku,job_name,channel_id
0,DE LONGHI,DE LONGHI Magnifica Evo espresso machine (ECAM...,1983043,Media market r.no.490 Serp_Job,58
1,SAGE,SAGE Oracle Touch Stainless Steel Espresso Mac...,1820038,Media market r.no.490 Serp_Job,58
2,DE LONGHI,DE LONGHI Magnifica Evo espresso machine (ECAM...,1983043,Media market r.no.490 Serp_Job,58
3,DE LONGHI,DE LONGHI Nespresso Lattissima One Evolution W...,1944067,Media market r.no.490 Serp_Job,58
4,SAECO,SAECO Saeco Xelsis Suprema espresso machine (S...,1983290,Media market r.no.490 Serp_Job,58
...,...,...,...,...,...
10927,KRUPS,KRUPS Dolce Gusto Genio S Plus (KP3408),1895845,Media market r.no.490 Serp_Job,58
10928,KRUPS,KRUPS Nescafé Dolce Gusto Infinissima Touch (K...,2000943,Media market r.no.490 Serp_Job,58
10929,KRUPS,Krups Dolce Gusto Genio S Touch (KP440E),1895846,Media market r.no.490 Serp_Job,58
10930,KRUPS,KRUPS KP270A10 I YY4653FD INFINISSIMA TOUCH GRAY,2000957,Media market r.no.490 Serp_Job,58


In [8]:
rawdf1=serp_details.drop_duplicates('sku')
print(len(rawdf1))
df_ASIN = rawdf1[["sku"]]
df_ASIN.dropna(inplace=True)
df_ASIN.drop_duplicates(inplace=True)
asinlist = list(df_ASIN['sku'])
len(asinlist)

403


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_157092\2397205133.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ASIN.dropna(inplace=True)
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_157092\2397205133.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ASIN.drop_duplicates(inplace=True)


403

In [9]:
query_2 = f""" 
Select channel_sku_id,country_code, brand_name, categories,category_id
from entity.product_master
where channel_sku_id in {tuple(asinlist)} 
"""

pm_data = pd.read_sql_query(query_2, conn[1])
pm_data

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_157092\3517360684.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pm_data = pd.read_sql_query(query_2, conn[1])


,channel_sku_id,country_code,brand_name,categories,category_id
0,1023329,BE,MELITTA,None,None
1,1043775,BE,PHILIPS,None,None
2,1085351,BE,MELITTA,None,None
3,1085587,BE,LAVAZZA,None,None
4,1090918,BE,MELITTA,None,None
...,...,...,...,...,...
401,2072619,BE,PHILIPS,None,None
402,2074875,BE,PHILIPS,None,None
403,9255045,BE,WARNER HOME VIDEO,None,None
404,9472157,BE,UNIVERSAL MUSIC,None,None


In [10]:
mapped_df=pd.merge(rawdf1,pm_data, how='left', left_on=['sku'],right_on=['channel_sku_id'])
mapped_df

,brand,title,sku,job_name,channel_id,channel_sku_id,country_code,brand_name,categories,category_id
0,DE LONGHI,DE LONGHI Magnifica Evo espresso machine (ECAM...,1983043,Media market r.no.490 Serp_Job,58,1983043,BE,DE LONGHI,None,None
1,SAGE,SAGE Oracle Touch Stainless Steel Espresso Mac...,1820038,Media market r.no.490 Serp_Job,58,1820038,BE,SAGE,None,None
2,DE LONGHI,DE LONGHI Nespresso Lattissima One Evolution W...,1944067,Media market r.no.490 Serp_Job,58,1944067,BE,DE LONGHI,None,None
3,SAECO,SAECO Saeco Xelsis Suprema espresso machine (S...,1983290,Media market r.no.490 Serp_Job,58,1983290,BE,SAECO,None,None
4,DE LONGHI,DE LONGHI Prestigo Black Espresso Machine (EC9...,1950023,Media market r.no.490 Serp_Job,58,1950023,BE,DE LONGHI,None,None
...,...,...,...,...,...,...,...,...,...,...
401,BOSE,BOSE Portable speaker Home Black (829393-2100),1843214,Media market r.no.490 Serp_Job,58,1843214,BE,BOSE,None,None
402,BOSE,BOSE Portable speaker Home Silver (829393-2300),1843217,Media market r.no.490 Serp_Job,58,1843217,BE,BOSE,None,None
403,JBL,JBL Boombox 3 Squad Portable Speaker (JBLBOOMB...,1996228,Media market r.no.490 Serp_Job,58,1996228,BE,JBL,None,None
404,MSI,MSI Optix G2412 24&quot; 170Hz Full-HD 1ms Gam...,2043569,Media market r.no.490 Serp_Job,58,2043569,BE,MSI,None,None


In [11]:
df2=mapped_df[mapped_df['categories'].isnull()][['channel_id','sku','title','brand_name']]
df2

,channel_id,sku,title,brand_name
0,58,1983043,DE LONGHI Magnifica Evo espresso machine (ECAM...,DE LONGHI
1,58,1820038,SAGE Oracle Touch Stainless Steel Espresso Mac...,SAGE
2,58,1944067,DE LONGHI Nespresso Lattissima One Evolution W...,DE LONGHI
3,58,1983290,SAECO Saeco Xelsis Suprema espresso machine (S...,SAECO
4,58,1950023,DE LONGHI Prestigo Black Espresso Machine (EC9...,DE LONGHI
...,...,...,...,...
401,58,1843214,BOSE Portable speaker Home Black (829393-2100),BOSE
402,58,1843217,BOSE Portable speaker Home Silver (829393-2300),BOSE
403,58,1996228,JBL Boombox 3 Squad Portable Speaker (JBLBOOMB...,JBL
404,58,2043569,MSI Optix G2412 24&quot; 170Hz Full-HD 1ms Gam...,MSI


In [12]:
df=pd.DataFrame(columns=['channel_sku_id','categories','seller_info','first_image'])
df

,channel_sku_id,categories,seller_info,first_image


In [13]:
asinlist = list(df2['sku'])
len(asinlist)

396

In [14]:
chrome_opt=Options()
chrome_opt.add_argument("--incognito")
driver = webdriver.Chrome(service=s,options=chrome_opt)
capabilities = driver.capabilities
chrome_driver_version = capabilities['chrome']['chromedriverVersion']

# Print the ChromeDriver version
print("ChromeDriver version:", chrome_driver_version)

ChromeDriver version: 114.0.5735.90 (386bc09e8f4f2e025eddae123f36f6263096ae49-refs/branch-heads/5735@{#1052})


In [24]:
for i in tqdm(range(366,len(asinlist))):
    sku=asinlist[i]
    url="https://www.mediamarkt.de/de/product/-{}.html".format(sku)
    
    print('************************')
    print(i)
    print(url)
    driver.get(url)
    first_image=''
    bs=BeautifulSoup(driver.page_source,'html.parser')
    cat=[]
    try:
        element = driver.find_elements(By.XPATH, "//ol[@class='Breadcrumb-styled__StyledList-sc-57676b39-1 cXCHDb']//li")
        cat=[element[e].text for e in range(1,len(element))] 
        print(cat)
    except:
         print('cat not  found')
    
    try:
        element = driver.find_element(By.XPATH, "//p[@class='sc-hLBbgP cWDBFq']")
        seller_info=element.text.strip()
        print(seller_info)
    except:
        print('seller not found')
    
    try:
        image = driver.find_element(By.XPATH, "//div[@class='ZoomImage__StyledZoomImage-sc-43dea761-0 gGEvAT']/picture/img")
        first_image=image.get_attribute('src')
        print(first_image)
    except:
        print('image not found')
    
        
    df1={'channel_sku_id': sku, 'categories': cat, 'first_image':first_image, 'seller_info':seller_info}
    df=df.append(df1, ignore_index = True)

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

************************
366
https://www.mediamarkt.de/de/product/-2035179.html


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_157092\1879312671.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df1, ignore_index = True)
  3%|██▊                                                                                | 1/30 [00:08<04:20,  9.00s/it]

['Film & Musik', 'Musik & Hörbücher', 'Vinyl/LP', 'Klassik']

https://assets.mmsrg.com/isr/166325/c1/-/pixelboxx-mss-69110344?x=480&y=334&format=jpg&quality=80&sp=yes&strip=yes&trim&ex=480&ey=334&align=center&resizesource&unsharp=1.5x1+0.7+0.02&cox=0&coy=0&cdx=480&cdy=334
************************
367
https://www.mediamarkt.de/de/product/-1944348.html


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_157092\1879312671.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df1, ignore_index = True)
  7%|█████▌                                                                             | 2/30 [00:17<04:06,  8.80s/it]

...

[]
seller not found
image not found
************************
374
https://www.mediamarkt.de/de/product/-1911029.html


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_157092\1879312671.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df1, ignore_index = True)
 30%|████████████████████████▉                                                          | 9/30 [01:03<02:42,  7.75s/it]

['Computer & Büro', 'PCs & Zubehör', 'Eingabegeräte', 'PC Mäuse']

https://assets.mmsrg.com/isr/166325/c1/-/pixelboxx-mss-67062243?x=480&y=334&format=jpg&quality=80&sp=yes&strip=yes&trim&ex=480&ey=334&align=center&resizesource&unsharp=1.5x1+0.7+0.02&cox=0&coy=0&cdx=480&cdy=334
************************
375
https://www.mediamarkt.de/de/product/-2066867.html


 30%|████████████████████████▉                                                          | 9/30 [01:22<03:12,  9.18s/it]


KeyboardInterrupt: 

In [25]:
df

,channel_sku_id,categories,seller_info,first_image
0,1983043,"[Spielzeug & Baby, Spielzeugfahrzeuge, Flugzeu...",,https://assets.mmsrg.com/isr/166325/c1/-/pixel...
1,1820038,[],,
2,1944067,"[Haushalt & Wohnen, Haushaltswaren, Küchenhelf...",,https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
3,1983290,"[Spielzeug & Baby, Spielzeugfahrzeuge, Flugzeu...","UVP 15,99\n€15,99",https://assets.mmsrg.com/isr/166325/c1/-/pixel...
4,1950023,[],"UVP 15,99\n€15,99",
5,2069110,"[Computer & Büro, PCs & Zubehör, All-in-One-PCs]",,https://assets.mmsrg.com/isr/166325/c1/-/pixel...
6,1992884,"[Film & Musik, Filme & Serien, Film Genres, Co...",,https://assets.mmsrg.com/isr/166325/c1/-/pixel...
7,2066353,"[Film & Musik, Musik & Hörbücher, Hörbücher & ...",,https://assets.mmsrg.com/isr/166325/c1/-/pixel...
8,1851653,"[Computer & Büro, PCs & Zubehör, PCs]",,https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
9,2031919,"[Computer & Büro, Drucker & Scanner, Papier, E...",,https://assets.mmsrg.com/isr/166325/c1/-/pixel...


In [26]:
final1=pd.merge(rawdf1,df,left_on='sku', right_on='channel_sku_id', how= 'right')
final=final1[['sku','title','brand','categories','first_image']]

In [27]:
final

,sku,title,brand,categories,first_image
0,1983043,DE LONGHI Magnifica Evo espresso machine (ECAM...,DE LONGHI,"[Spielzeug & Baby, Spielzeugfahrzeuge, Flugzeu...",https://assets.mmsrg.com/isr/166325/c1/-/pixel...
1,1820038,SAGE Oracle Touch Stainless Steel Espresso Mac...,SAGE,[],
2,1944067,DE LONGHI Nespresso Lattissima One Evolution W...,DE LONGHI,"[Haushalt & Wohnen, Haushaltswaren, Küchenhelf...",https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
3,1983290,SAECO Saeco Xelsis Suprema espresso machine (S...,SAECO,"[Spielzeug & Baby, Spielzeugfahrzeuge, Flugzeu...",https://assets.mmsrg.com/isr/166325/c1/-/pixel...
4,1950023,DE LONGHI Prestigo Black Espresso Machine (EC9...,DE LONGHI,[],
5,2069110,SIEMENS Front washing machine iQ500 A (WG44G2A...,SIEMENS,"[Computer & Büro, PCs & Zubehör, All-in-One-PCs]",https://assets.mmsrg.com/isr/166325/c1/-/pixel...
6,1992884,PHILIPS Handheld steamer 3000 Series (STH3010/70),PHILIPS,"[Film & Musik, Filme & Serien, Film Genres, Co...",https://assets.mmsrg.com/isr/166325/c1/-/pixel...
7,2066353,PHILIPS PerfectCare Elite steam generator iron...,PHILIPS,"[Film & Musik, Musik & Hörbücher, Hörbücher & ...",https://assets.mmsrg.com/isr/166325/c1/-/pixel...
8,1851653,PHILIPS Shaver Series 3000 (S3233/52),PHILIPS,"[Computer & Büro, PCs & Zubehör, PCs]",https://assets.mmsrg.com/isr/166325/c1/-/ASSET...
9,2031919,PHILIPS OneBlade Pro beard trimmer (QP6541/15),PHILIPS,"[Computer & Büro, Drucker & Scanner, Papier, E...",https://assets.mmsrg.com/isr/166325/c1/-/pixel...


In [ ]:
request_no=input("Enter the request number of the requests seperated with underscore(_) ")
#547_552
file='final_output_mediamarket_be_{}.xlsx'.format(request_no)

In [ ]:
final.to_excel(file)

In [ ]:
email_user = 'alerts@1digitalstack.ai'
email_password = ''
email_send = ['ghansham.pindoriya@1digitalstack.ai','karan.kumar@1digitalstack.ai','arihant.singh@1digitalstack.ai','sambhav.jain@1digitalstack.ai','ankur.dwivedi@1digitalstack.ai','kinshu@1digitalstack.ai']

subject = file
msg = MIMEMultipart()
msg['From'] = email_user
#msg['From'] = "'Abhishek' <{email_user}>"
msg['To'] = ', '.join(email_send)
msg['Subject'] = subject

body = 'Hi,\n\nPFA of today''s mapping request.Please reply as done as you pick this job.'
msg.attach(MIMEText(body,'plain'))

filename=[file]

for file in filename:
    attachment  =open(file,'rb')
    part = MIMEBase('application','octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= "+file)
    msg.attach(part)
    text = msg.as_string()

server = smtplib.SMTP('1digitalstack.com',25)
#server.login(email_user,email_password)

server.sendmail(email_user,email_send,text)
server.quit()